In [1]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers

import nltk
from nltk.corpus import wordnet as wn
import inflect

from keras.models import Sequential
from keras import layers
import numpy as np
from collections import Counter, defaultdict

from gensim.utils import tokenize
from itertools import groupby

from keras.models import Input, Model
from keras.layers import Dense, Dropout
from keras.layers import LSTM, RepeatVector
from keras.layers.wrappers import TimeDistributed

Using TensorFlow backend.


In [2]:
p = inflect.engine()

In [3]:
pairs = {}
for synset in wn.all_synsets('n'):
    word = synset.name().split('.', 1)[0]
    if not word in pairs:
        pairs[word] = p.plural(word)
len(pairs)

67176

In [4]:
with open('data/plurals.txt', 'w') as fout:
    for k in sorted(pairs):
        if '_' in k or '-' in k:
            continue
        if k.isdigit():
            continue
        fout.write('%s\t%s\n' % (k, pairs[k]))

In [5]:
p.plural('no')

'noes'

In [6]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [7]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


In [8]:
# Parameters for the model and dataset.
INVERT = True

In [9]:
questions = []
expected = []
seen = set()
#with open('data/en_de.txt') as fin:
with open('data/plurals.txt') as fin:
    for line in fin:
        en, de = line.strip().split('\t')
        questions.append(en)
        expected.append(de)

max_question_len = max(len(q) for q in questions)
max_expected_len = max(len(e) for e in expected)
questions = [' ' * (max_question_len - len(q)) + q for q in questions]
expected = [e + ' ' * (max_expected_len - len(e)) for e in expected]
if INVERT:
    questions = [q[::-1] for q in questions]

print('Total addition questions:', len(questions))

Total addition questions: 39929


In [10]:
chars = set(ch for k, v in zip(questions, expected) for ch in k + v)
ctable = CharacterTable(chars)
len(chars)

40

In [11]:
print('Vectorization...')
x = np.zeros((len(questions), max_question_len, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), max_expected_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, max_question_len)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, max_expected_len)
print('done')

Vectorization...
done


In [12]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)


Training Data:
(35937, 31, 40)
(35937, 32, 40)
Validation Data:
(3992, 31, 40)
(3992, 32, 40)


In [13]:
# The below is taken from: https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py
RNN = layers.LSTM
HIDDEN_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(max_question_len, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
#model.add(layers.Dropout(DROP_OUT))
model.add(layers.RepeatVector(max_expected_len))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
#    model.add(layers.Dropout(DROP_OUT))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               86528     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 32, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32, 128)           131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 40)            5160      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 40)            0         
Total params: 223,272
Trainable params: 

In [19]:
def create_seq2seq(num_nodes, num_layers):
    num_chars = len(chars)
    question = Input(shape=(max_question_len, num_chars), name='question')
    # repeat = RepeatVector(max_expected_len)(question)
    prev = question
    for _ in range(num_layers):
        lstm = LSTM(num_nodes, return_sequences=True, name='lstm_layer_%d' % (i + 1))(prev)
        prev = lstm
    dense = TimeDistributed(Dense(num_chars, name='dense', activation='softmax'))(prev)
    model = Model(inputs=[input], outputs=[dense])
    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

seq2seq = create_seq2seq(128, 1)

TypeError: Input tensors to a Model must be Keras tensors. Found: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7f749a94ce48>> (missing Keras metadata).

In [17]:

BATCH_SIZE = 2048

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=10,
              validation_data=(x_val, y_val))
    print()
    print('-' * 50)
    print('Iteration', iteration)
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print(q[::-1] if INVERT else q, '(%s)' % correct, '-', guess)


Train on 35937 samples, validate on 3992 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[2048,32] and type int64 on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node metrics/acc/ArgMax (defined at /home/ohtamans/.pyenv/versions/3.6.7/envs/cookbook/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1392)  = ArgMax[T=DT_FLOAT, Tidx=DT_INT32, output_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_activation_1_target_0_1/_61, metrics/acc/ArgMax/dimension)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss/mul/_97}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3292_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see 

In [ ]:
shakespeare = strip_headers(load_etext(100))
tokens = [tuple(word) for word in tokenize(plays, to_lower=True)]
token_counts = Counter(tokens)

In [ ]:
pairs = [(token[i], token[i + 1], token_id) for token_id, token in enumerate(tokens) for i in range(len(token) - 1)]

In [ ]:
pairs[10]

In [ ]:
help(tokenize)

In [ ]:
tokens[5]